# Integration with R

This tutorial illustrates how to use **pyam** features
in an **R workflow** using the [reticulate package](https://rstudio.github.io/reticulate/).

<img style="float: right; margin: 10px;" src="_static/R.png">

The source code of the tutorial is available in the folder
[doc/source/R_tutorials](https://github.com/IAMconsortium/pyam/tree/master/doc/source/R_tutorials)
of the **pyam** GitHub repository.

## Installation and requirements

Assuming you have a working and current R installation,
you can follow the steps below to install the requirements to run this tutorial:

0. Install Python using [Anaconda](https://www.anaconda.com)  
   *This is the recommended approach for users that aren't already experienced with Python...*  


1. Install **pyam** ([read the docs](https://pyam-iamc.readthedocs.io/en/stable/install.html)
   for more information)  
   ```
   conda install -c conda-forge pyam
   ```

2. To use R in a Jupyter notebook as in this tutorial,
   install Jupyter and the [IRkernel](https://irkernel.github.io) via conda  
   ```
   conda install jupyter ir-kernel
   ```

3. In R, install the **reticulate** package from CRAN  
   ``` r
   install.packages("reticulate")
   ```

<div class="alert alert-warning">

This tutorial is an illustration on how to get started,
not a full-fledged documentation of a **pyam-R** interface...  
The integration between Python and R using **reticulate** is a bit fickle -
having multiple Python installations/enviroments
or outdated versions of **reticulate**/**pandas** can cause issues.  
Please [read the docs](https://rstudio.github.io/reticulate/) if you experience any problems!

</div>

This notebook was run with R version 4.0 and Python 3.8 on Mac OS.

****

**Developers note:**
Running this notebook on CI and RTD is currently not supported.  
For the time being, this notebook is not executed by **nbsphinx**
and has to be saved with output.

## Setting up the session

In [1]:
require(reticulate)

Loading required package: reticulate



The next line instructs **reticulate** to use the conda 'base' environment.  
[Read the docs](https://rstudio.github.io/reticulate/reference/use_python.html)
in case you want to use an alternative installation or environment!

In [2]:
use_condaenv()

Now, we import **pyam** to the R session.

In [3]:
pyam <- import("pyam")

## Passing an R dataframe to pyam

The first cell of this section creates a simple timeseries data table
following the IAMC format as an R dataframe.

In [4]:
data <- data.frame(
  model=c('model_a', 'model_a', 'model_a'),
  scenario=c('scen_a', 'scen_a', 'scen_b'),
  region=c('World', 'World', 'World'),
  variable=c('Primary Energy', 'Primary Energy|Coal', 'Primary Energy'),
  unit=c('EJ/yr', 'EJ/yr', 'EJ/yr'),
  "2005"=c(1, 0.5, 2),
  "2010"=c(6, 3, 7)
)
data

model,scenario,region,variable,unit,X2005,X2010
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
model_a,scen_a,World,Primary Energy,EJ/yr,1.0,6
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,0.5,3
model_a,scen_b,World,Primary Energy,EJ/yr,2.0,7


The following cell casts the data to a **pyam.IamDataFrame**.

<div class="alert alert-info">

When migrating code from **Python** to **R**, keep in mind that  
you have to use `$` instead of `.` to call an object's methods!

</div>

In [5]:
df <- pyam$IamDataFrame(data)
df

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : model_a (1)
 * scenario : scen_a, scen_b (2)
Timeseries data coordinates:
   region   : World (1)
   variable : Primary Energy, Primary Energy|Coal (2)
   unit     : EJ/yr (1)
   year     : 2005, 2010 (2)
Meta indicators:
   exclude (bool) False (1)

*Ending a cell of a Jupyter notebook with the **IamDataFrame** instance
shows the same overview as if you were in a Python environment!*

The last cell of this section retrieves the timeseries data from the **IamDataFrame** in long form
and returns it again as an **R data.frame**.

In [6]:
df$data

model,scenario,region,variable,unit,year,value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
model_a,scen_a,World,Primary Energy,EJ/yr,2005,1.0
model_a,scen_a,World,Primary Energy,EJ/yr,2010,6.0
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,2005,0.5
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,2010,3.0
model_a,scen_b,World,Primary Energy,EJ/yr,2005,2.0
model_a,scen_b,World,Primary Energy,EJ/yr,2010,7.0


## Calling a pyam method

For illustration, we will use the [interpolate()](https://pyam-iamc.readthedocs.io/en/stable/api/iamdataframe.html#pyam.IamDataFrame.interpolate) method
on the **IamDataFrame** instance.

<div class="alert alert-warning">

One key difference between **R** and **Python** are the default types:  
whereas Python will interpret "2007" as an integer, R will treat the same input as a float.  
In the example below, we have to explicitly instruct R to pass an integer to the **pyam** function.

</div>

In [7]:
x <- df$interpolate(as.integer(2007), inplace=TRUE)

In [8]:
df$data

model,scenario,region,variable,unit,year,value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
model_a,scen_a,World,Primary Energy,EJ/yr,2005,1.0
model_a,scen_a,World,Primary Energy,EJ/yr,2007,3.0
model_a,scen_a,World,Primary Energy,EJ/yr,2010,6.0
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,2005,0.5
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,2007,1.5
model_a,scen_a,World,Primary Energy|Coal,EJ/yr,2010,3.0
model_a,scen_b,World,Primary Energy,EJ/yr,2005,2.0
model_a,scen_b,World,Primary Energy,EJ/yr,2007,4.0
model_a,scen_b,World,Primary Energy,EJ/yr,2010,7.0


## Query data from an IIASA data resource

The IIASA *Energy, Climate, and Environment* Program hosts a suite of **Scenario Explorer** instances and related infrastructure to support analysis of integrated-assessment pathways in IPCC reports and model comparison projects. 
High-profile use cases include the [IAMC 1.5°C Scenario Explorer hosted by IIASA](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer) supporting the *IPCC Special Report on Global Warming of 1.5°C* (SR15) and the Horizon 2020 project [CD-LINKS](https://data.ene.iiasa.ac.at/cd-links).

The **pyam** package can retrieve scenario data directly from any **Scenario Explorer** database instance
hosted by IIASA for use in your processing and analysis workflows.

In [9]:
df = pyam$read_iiasa(
    'iamc15',
    model='MESSAGEix*', 
    variable=c('Emissions|CO2', 'Primary Energy|Coal'), 
    region='World',
    meta=c('category')
)

In [10]:
df

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : MESSAGEix-GLOBIOM 1.0 (1)
 * scenario : CD-LINKS_INDCi, CD-LINKS_NPi, CD-LINKS_NPi2020_1000, ... LowEnergyDemand (7)
Timeseries data coordinates:
   region   : World (1)
   variable : Emissions|CO2, Primary Energy|Coal (2)
   unit     : EJ/yr, Mt CO2/yr (2)
   year     : 2000, 2005, 2010, 2020, 2030, 2040, 2050, 2060, ... 2100 (12)
Meta indicators:
   version (object) 1 (1)
   carbon price|Avg NPV (2030-2100) (object) 0.9123249940000001, ... 30.1503013256 (7)
   year of netzero CO2 emissions (object) nan, 2065, 2078, 2053, 2060.0 (5)
   carbon price|2100 (NPV) (object) 0.0426419715, 0.012143570900000001, ... 14.2051784281 (7)
   cumulative CO2 emissions (2016 to peak warming, Gt CO2) (object) ... (7)
   ...

See the [pyam-IIASA-database tutorial](https://pyam-iamc.readthedocs.io/en/stable/tutorials/iiasa_dbs.html)
or the [API documentation](https://pyam-iamc.readthedocs.io/en/stable/api/iiasa.html)
for more information and a complete list of features!

## Next steps

<div class="alert alert-info">
    
**New to Jupyter notebooks?**  
Read [this page](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/) for helpful tips and tricks when working with Jupyter notebooks.

</div>

### Questions?

Take a look at the other tutorials to see the scope of **pyam** features - then join our [mailing list](https://groups.io/g/pyam)!

### Problems?

If you encounter any functions or methods that don't work as expected,
please check whether there are already any [issues](https://github.com/IAMconsortium/pyam/labels/R) in our GitHub repo.  
If not, start a [new one](https://github.com/IAMconsortium/pyam/issues/new)!